[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[python-ecdsa docs](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [3]:
from ecdsa import SigningKey, SECP256k1

private_key = SigningKey.generate(curve=SECP256k1)
public_key = private_key.get_verifying_key()

message = b"I need coffee"
signature = private_key.sign(message)

In [4]:
public_key.verify(signature, message)

True

In [5]:
public_key.verify(signature, b"Im tired")

BadSignatureError: 

# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. 
    * Where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spend the coin append a new transfer. Use the public key of the person you are sending to, and sign it using your private key.

In [6]:
class Transfer:
    def __init__(self, signature, public_key):
        self.signature = signature
        self.public_key = public_key
        
class EDSACoin:
    def __init__(self, transfers):
        self.transfers = transfers

In [10]:
# The usual suspects ... 
# SECP256k1 is a detail about the "magical multiplication" used under the covers

bank_private_key = SigningKey.generate(curve=SECP256k1)
bob_private_key = SigningKey.generate(curve=SECP256k1)
alice_private_key = SigningKey.generate(curve=SECP256k1)

bank_public_key = bank_private_key.get_verifying_key()
bob_public_key = bob_private_key.get_verifying_key()
alice_public_key = alice_private_key.get_verifying_key()

In [7]:
from utils import serialize

def issue(public_key):
    message = serialize(public_key)
    
    signature = bank_private_key.sign(message)    
    transfer = Transfer(
        signature=signature,
        public_key=public_key
    )

    coin = EDSACoin([transfer])
    return coin

# Validating the First Transfer

In [8]:
def validate(coin):
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    bank_public_key.verify(transfer.signature, message)

In [11]:
alice_coin = issue(alice_public_key)

validate(alice_coin)

In [12]:
message = serialize(bob_public_key)
signature = bob_private_key.sign(message)    
transfer = Transfer(
    signature=signature,
    public_key=bob_public_key
)
alt_coin = EDSACoin([transfer])

In [13]:
from ecdsa import BadSignatureError

try:
    validate(alt_coin)
except BadSignatureError:
    print("Bad signature detected")

Bad signature detected


# Validating Subsequent Transfers

In [14]:
def transfer_message(previous_signature, next_owner_public_key):
    return serialize({
        "previous_signature": previous_signature,
        "next_owner_public_key": next_owner_public_key
    })

def validate(coin):
    # Check the first transfer
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    bank_public_key.verify(transfer.signature, message)
    
    # Check the rest of coin.transfers
    previous_transfer = coin.transfers[0]
    for next_transfer in coin.transfers[1:]:
        message = transfer_message(previous_transfer.signature, next_transfer.public_key)
        previous_transfer.publick_key.verify(
            next_transfer.signature,
            message
        )
        

In [20]:
def get_owner(coin):
    database = {
        serialize(bob_public_key): "Bob",
        serialize(alice_public_key): "Alice",
        serialize(bank_public_key): "Bank",
    }
    public_key = serialize(coin.transfers[-1].public_key)
    return database[public_key]

In [22]:
coin = issue(alice_public_key)
print("This coin is owned by ", get_owner(coin))

message = transfer_message(coin.transfers[-1].signature, bob_public_key)
alice_to_bob = Transfer(
    signature = alice_private_key.sign(message),
    public_key = bob_public_key
)
coin.transfers.append(alice_to_bob)
print("This coin is owned by ", get_owner(coin))

message = transfer_message(coin.transfers[-1].signature, bank_public_key)
bob_to_bank = Transfer(
    signature = bob_private_key.sign(message),
    public_key = bank_public_key
)
coin.transfers.append(bob_to_bank)
print("This coin is owned by ", get_owner(coin))

This coin is owned by  Alice
This coin is owned by  Bob
This coin is owned by  Bank


# Serialization

In [23]:
from utils import to_disk, from_disk

In [24]:
import os

filename = "coin.edcsacoin"

print("Does the coin file exist on disk?", os.path.isfile(filename))

Does the coin file exist on disk? False


In [26]:
coin = issue(alice_public_key)
to_disk(coin, filename)

In [27]:
print("Does the coin file exist on disk?", os.path.isfile(filename))

Does the coin file exist on disk? True


In [28]:
coin = from_disk(filename)
coin

# The Finished Product

[ecdsacoin.py](ecdsacoin.py)

In [21]:
import ecdsacoin

coin = ecdsacoin.issue(alice_public_key)
coin.validate()

alice_to_bob = Transfer(
    signature=alice_private_key.sign(transfer_message(coin.transfers[-1].signature, bob_public_key)),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()